## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Cameta,BR,-2.2444,-49.4958,89.06,53,42,4.07,scattered clouds
1,1,Bluff,NZ,-46.6000,168.3333,54.70,85,6,1.74,clear sky
2,2,Chinsali,ZM,-10.5414,32.0816,71.71,88,69,2.04,moderate rain
3,3,Punta Arenas,CL,-53.1500,-70.9167,51.80,50,40,18.41,scattered clouds
4,4,Khatanga,RU,71.9667,102.5000,-36.76,78,0,5.28,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Cameta,BR,-2.2444,-49.4958,89.06,53,42,4.07,scattered clouds
1,1,Bluff,NZ,-46.6000,168.3333,54.70,85,6,1.74,clear sky
2,2,Chinsali,ZM,-10.5414,32.0816,71.71,88,69,2.04,moderate rain
3,3,Punta Arenas,CL,-53.1500,-70.9167,51.80,50,40,18.41,scattered clouds
5,5,Chaumont,FR,48.1167,5.1333,48.00,93,100,11.01,overcast clouds
10,10,Saint-Philippe,RE,-21.3585,55.7679,84.20,70,75,6.91,light rain
12,12,Avarua,CK,-21.2078,-159.7750,77.00,88,20,3.44,light rain
13,13,Vaini,TO,-21.2000,-175.2000,77.00,100,75,4.61,light rain
14,14,Busselton,AU,-33.6500,115.3333,60.01,93,100,3.00,light rain
16,16,Chuy,UY,-33.6971,-53.4616,77.11,54,0,10.02,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        462
City           462
Country        460
Lat            462
Lng            462
Max Temp       462
Humidity       462
Cloudiness     462
Wind Speed     462
Description    462
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')
clean_df.count()

City_ID        460
City           460
Country        460
Lat            460
Lng            460
Max Temp       460
Humidity       460
Cloudiness     460
Wind Speed     460
Description    460
dtype: int64

In [10]:
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Cameta,BR,-2.2444,-49.4958,89.06,53,42,4.07,scattered clouds
1,1,Bluff,NZ,-46.6000,168.3333,54.70,85,6,1.74,clear sky
2,2,Chinsali,ZM,-10.5414,32.0816,71.71,88,69,2.04,moderate rain
3,3,Punta Arenas,CL,-53.1500,-70.9167,51.80,50,40,18.41,scattered clouds
5,5,Chaumont,FR,48.1167,5.1333,48.00,93,100,11.01,overcast clouds


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Cameta,BR,89.06,scattered clouds,-2.2444,-49.4958,
1,Bluff,NZ,54.70,clear sky,-46.6000,168.3333,
2,Chinsali,ZM,71.71,moderate rain,-10.5414,32.0816,
3,Punta Arenas,CL,51.80,scattered clouds,-53.1500,-70.9167,
5,Chaumont,FR,48.00,overcast clouds,48.1167,5.1333,
10,Saint-Philippe,RE,84.20,light rain,-21.3585,55.7679,
12,Avarua,CK,77.00,light rain,-21.2078,-159.7750,
13,Vaini,TO,77.00,light rain,-21.2000,-175.2000,
14,Busselton,AU,60.01,light rain,-33.6500,115.3333,
16,Chuy,UY,77.11,clear sky,-33.6971,-53.4616,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Cameta,BR,89.06,scattered clouds,-2.2444,-49.4958,Hotel San Juan
1,Bluff,NZ,54.70,clear sky,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
2,Chinsali,ZM,71.71,moderate rain,-10.5414,32.0816,Mwiluzi Motel Chinsali Ltd
3,Punta Arenas,CL,51.80,scattered clouds,-53.1500,-70.9167,Hotel Dreams Del Estrecho
5,Chaumont,FR,48.00,overcast clouds,48.1167,5.1333,Hôtel Restaurant Des Remparts


In [15]:
hotel_df.count()
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df.count()

City           460
Country        460
Max Temp       460
Description    460
Lat            460
Lng            460
Hotel Name     460
dtype: int64

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
clean_hotel_df = 

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

'0.8.0'

Known nbextensions:

      - Validating: ok



  config dir: C:\Users\ARCHANANarula\anaconda3\etc\jupyter\nbconfig
    notebook section
      jupyter-js-widgets/extension enabled 
